In [1]:
print("apple")

apple


In [5]:
from scipy.io import wavfile

rate,x = wavfile.read("wavs/twinkle_twinkle.wav")
return 

ValueError: invalid literal for int() with base 10: ''

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.io import wavfile
from IPython.display import display, Audio

sns.set(font_scale=1.4)

class Sound:
    """ a sound, can be played, volume adjusted or appended

    Attributes:
        x (np.array): an array representing the sound. either
            1d (mono) or 2d (stereo) (see note above)
        rate (int): frequency, in hertz, that the signal x
            should be played at
    """

    def __init__(self, x, rate):
        self.x = x.astype(np.int16)
        self.rate = rate

    @classmethod
    def from_wav_file(cls, file):
        """ loads sound from a Wav file
        
        Args:
            file (str): file name (needn't have full path if
                file is in same folder as this ipynb)
        """
        rate, x = wavfile.read(str(file))
        return Sound(x=x, rate=rate)
    
    def to_wav_file(self, file):
        """ save sound to a Wav file
        
        Args:
            file (str): file name (needn't have full path if
                file is in same folder as this ipynb)
        """
        wavfile.write(file, self.rate, self.x)

    def build_jupyter_player(self):
        """ builds and displays jupyter player of sound """
        display(Audio((self.x[:, 0], self.x[:, 1]), 
                      rate=self.rate))
        
    def __mul__(self, other):
        """multiplies the audio signal by some constant
        
         Args:
            other (int or float): some scale to apply
        """
        assert float(other) >= 0, \
            'Sound can only be multiplied by positive values'
        assert type(other) in (int, float), \
            'Sound can only be multiplied by int or float'
        
        return Sound(x=self.x * other, rate=self.rate)

    def __and__(self, other):
        """appends one noise onto the end of another
        
        Args:
            other (obj): sound to append to original sound
        """
        assert self.rate == other.rate, \
            'Only Sound arrays of same rate can be appended'
        
        new_sound = np.append(self.x, other.x, axis=0)
        return Sound(x=new_sound, rate=self.rate)
    
    def __add__(self, other):
        """overlays one sound on top of another
        
         Args:
            other (obj): sound to overlay atop original sound
        """
        assert self.rate == other.rate, \
            'Only Sound arrays of same rate can be overlayed'
        
        # list to hold both arrays and array shapes
        array_ls = [self.x, other.x]
        array_sizes = [np.shape(self.x), np.shape(other.x)]
        
        # finding which is the smaller and larger array (if any)
        larger_array = max(array_sizes)
        smaller_array = min(array_sizes)
        
        # finding index of the smaller and larger array for easy access
        smaller_idx = array_sizes.index(smaller_array)
        larger_idx = array_sizes.index(larger_array)
        
        # calculates how much bigger the larger array
        difference = larger_array[0] - smaller_array[0]
        
        # appending array of zeroes to fill the difference amount between the arrays
        new_array = np.append(array_ls[smaller_idx], np.zeros((difference, 2)), axis = 0)
        
        # overlaying the arrays
        combined_array = array_ls[larger_idx] + new_array
        
        return Sound(x=combined_array, rate=self.rate)
    
    def plot(self):
        """produces a graph of the audio signal"""
        
        # list to hold all time indexes in length of array
        time_ls = []
        for idx in range(len(self.x)):
            time_ls.append(idx)
            
        # plotting right and left audio sides
        plt.plot(time_ls, self.x[:, 0], label = "left")
        plt.plot(time_ls, self.x[:, 1], color = "darkorange", label = "right")
            
        # labeling plot and legend
        plt.xlabel("time")
        plt.ylabel("signal")
        plt.legend()
        

In [ ]:
thud = Sound.from_wav_file(file='sound_wav/thud_STEREO.wav')

thud.build_jupyter_player()